In [71]:
from dataclasses import dataclass
from abc import ABC, abstractmethod
import numpy as np
from typing import List

class ActivationFunction(ABC):
    @abstractmethod
    def function(self):
        pass

    @abstractmethod
    def derivative(self):
        pass


In [73]:

@dataclass
class Sigmoid(ActivationFunction):
    name: str = 'sigmoid'

    def function(self, x):
        return 1/(1 + np.exp(-x))
    
    def derivative(self, x):
        return self.function(x)*(1-self.function(x))

@dataclass
class Neuron():
    input_shape: int
    activation: ActivationFunction
    weights: np.ndarray[np.float64] = None
    bias: float = np.random.rand()

    def __post_init__(self):
        self.weights = np.random.rand(self.input_shape)

    def forward(self, x):
        weighted_sum = np.dot(self.weights, x) + self.bias
        return self.activation.function(weighted_sum)
    
@dataclass
class Layer():
    num_neurons: int
    input_tensor: np.ndarray
    activation: ActivationFunction
    neurons: List[Neuron] = None

    def __post_init__(self):
        self.neurons = [Neuron(np.shape(self.input_tensor), self.activation) for _ in range(self.num_neurons)]

In [75]:
X = [[0, 0], [0, 1], [1, 0], [1, 1]]
Y = [0, 1, 1, 1]

In [79]:
n1 = Neuron(2, Sigmoid())

In [93]:
lr = 0.05

for i in range(100000):
    del_w0 = 0
    del_b0 = 0
    for x, y in zip(X, Y):
        a0 = n1.forward(np.array(x))

        delta = (a0 - Y[0]) * Sigmoid().derivative(np.array(X[0]))
        del_w0 += np.dot(delta, np.transpose(x))
        del_b0 += delta

    

In [95]:
delta_d

array([0.1448286, 0.1448286])

In [88]:
Sigmoid().derivative(np.array(X[0]))

array([0.25, 0.25])

In [90]:
delta = n1.forward(X[0]) - Y[0] * Sigmoid().derivative(np.array(X[0]))

[0.57931439 0.57931439]
